
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


### Prepare dataset ###

#### Connect to Google Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract


In [5]:
%cd /content/drive/My Drive/Datasets/
%cp unmasked-dataset.zip /content/
%cp masked-dataset.zip /content/
%cd /
!unzip -o /content/unmasked-dataset.zip 
!unzip -o /content/masked-dataset.zip 
%cd /content/

/content/drive/My Drive/Datasets
/
Archive:  /content/unmasked-dataset.zip
  inflating: content/training_demo/images/4JOL4C9BVQ.xml  
  inflating: content/training_demo/images/3DD6FSJ21S.xml  
  inflating: content/training_demo/images/1ALR08DTAA.xml  
  inflating: content/training_demo/images/0ZJLIECDHZ.jpg  
  inflating: content/training_demo/images/2PG6UOO49R.xml  
  inflating: content/training_demo/images/0TZET4XDJ3.xml  
  inflating: content/training_demo/images/2PWNQYGY5L.xml  
  inflating: content/training_demo/images/0DD4NVKTZF.jpg  
  inflating: content/training_demo/images/1T5OD2SUM1.xml  
  inflating: content/training_demo/images/2W6L6NNTWZ.xml  
  inflating: content/training_demo/images/3B96Y0Z8NA.jpg  
  inflating: content/training_demo/images/2WA7O2SG0J.xml  
  inflating: content/training_demo/images/4OQDM7Y542.xml  
  inflating: content/training_demo/images/0AMNAPIFEX.jpg  
  inflating: content/training_demo/images/3DHCZXWSD8.xml  
  inflating: content/training_demo/image

## Partitioning the images

We continue our data preparation by splitting our dataset into training and test sets.

Typically, the ratio is 90%/10%, i.e. 90% of the images are used for training and the rest 10% is maintained for testing, but you can chose whatever ratio suits your needs.


### Import necessary libraries

In [0]:
import os
import re
import shutil
from PIL import Image
import shutil
import glob
import math
import random
import pandas as pd
import xml.etree.ElementTree as ET

#### Define folder locations and split ratio

In [7]:
imageDir = os.getcwd() + "/training_demo/images"
print(imageDir)
outputDir = imageDir 
ratio = 0.1

/content/training_demo/images


### Partitioning the images

In [9]:
def iterate_dir(source, dest, ratio):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    print(train_dir)
    test_dir = os.path.join(dest, 'test')
    print(test_dir)
    
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

        
    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

     
    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)
    
    print(num_test_images)
        
    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        
        filename = images[idx]
        filename_path = os.path.join(source, filename)
        
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        xml_path = os.path.join(source, xml_filename)
        
        #copy image + XML files across only if accompanying XML file exists
        if os.path.isfile(xml_path):     
                shutil.copyfile(filename_path, os.path.join(test_dir, filename))
                shutil.copyfile(xml_path,os.path.join(test_dir,xml_filename))
        else:
            print("WARNING: the following XML file does not exist:" + xml_path)
        images.remove(images[idx])

    for filename in images:
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        xml_path = os.path.join(source, xml_filename)
        filename_path = os.path.join(source, filename)
        
        #copy image + XML files across only if accompanying XML file exists
        if os.path.isfile(xml_path):
                shutil.copyfile(filename_path, os.path.join(train_dir, filename))
                shutil.copyfile(xml_path, os.path.join(train_dir, xml_filename))
        else:
            print("WARNING: the following XML file does not exist:" + xml_path)

iterate_dir(imageDir, outputDir, ratio)

/content/training_demo/images/train
/content/training_demo/images/test
191


#### Delete original files (Optional) 

Once we are happy the images have successfully split into training and test sets, we can go ahead and delete the original images.

In [10]:
!rm -v /content/training_demo/images/*

removed '/content/training_demo/images/0004STET6P.jpg'
removed '/content/training_demo/images/0004STET6P.xml'
removed '/content/training_demo/images/000N7AIAFT.jpg'
removed '/content/training_demo/images/000N7AIAFT.xml'
removed '/content/training_demo/images/00H858UYSD.jpg'
removed '/content/training_demo/images/00H858UYSD.xml'
removed '/content/training_demo/images/00KPGHV40E.jpg'
removed '/content/training_demo/images/00KPGHV40E.xml'
removed '/content/training_demo/images/00P2LUXJW3.jpg'
removed '/content/training_demo/images/00P2LUXJW3.xml'
removed '/content/training_demo/images/00PYA83V1P.jpg'
removed '/content/training_demo/images/00PYA83V1P.xml'
removed '/content/training_demo/images/00SD82OK2A.jpg'
removed '/content/training_demo/images/00SD82OK2A.xml'
removed '/content/training_demo/images/00TMYXA5DF.jpg'
removed '/content/training_demo/images/00TMYXA5DF.xml'
removed '/content/training_demo/images/00V5CZZSSO.jpg'
removed '/content/training_demo/images/00V5CZZSSO.xml'
removed '/

## Label Map

TensorFlow requires a label map, which namely maps each of the used labels to an integer values. This label map is used both by the training and detection processes.

In [11]:
!mkdir -p training_demo/annotations
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/label_map.pbtxt -P training_demo/annotations

--2020-06-04 15:24:42--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [text/plain]
Saving to: ‘training_demo/annotations/label_map.pbtxt’

label_map.pbtxt     100%[===================>]      79  --.-KB/s    in 0s      

2020-06-04 15:24:42 (4.02 MB/s) - ‘training_demo/annotations/label_map.pbtxt’ saved [79/79]



## XML to CSV ##

And finally we need to generate a CSV file containing all image detail / classes from the individual XML files.

### Define locations for data

In [0]:
dataset_path = os.getcwd() + "/training_demo"
images_path = dataset_path + "/images"
annotations_path = dataset_path + "/annotations"


### Code to generate CSV files

In [0]:
def xml_to_csv(path):
    xml_list = []
    print("test1"+path)
    for xml_file in glob.glob(path + '/*.xml'):
        print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
            
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def generate_csv(folder):
    xml_path = os.path.join(images_path , folder)
    print(xml_path)
    xml_df = xml_to_csv(xml_path)
    print(xml_df)
    xml_df.to_csv(annotations_path + '/' + folder +'_labels.csv', index=None)
    csv_path = dataset_path + "/" + folder
    print('Successfully converted train xml to csv.')

#### Generate CSV files

For the train and test folders and place result in annotations folder

In [19]:
generate_csv("train")
generate_csv("test")

/content/training_demo/images/train
test1/content/training_demo/images/train
/content/training_demo/images/train/4JOL4C9BVQ.xml
/content/training_demo/images/train/9V1Y7JYQNF.xml
/content/training_demo/images/train/3DD6FSJ21S.xml
/content/training_demo/images/train/1ALR08DTAA.xml
/content/training_demo/images/train/K9PFT0A194.xml
/content/training_demo/images/train/2PG6UOO49R.xml
/content/training_demo/images/train/WAKUO7V02E.xml
/content/training_demo/images/train/0TZET4XDJ3.xml
/content/training_demo/images/train/KA4FMGBO8H.xml
/content/training_demo/images/train/2PWNQYGY5L.xml
/content/training_demo/images/train/9VFXN07J13.xml
/content/training_demo/images/train/J9KV0D4ZLL.xml
/content/training_demo/images/train/1T5OD2SUM1.xml
/content/training_demo/images/train/2W6L6NNTWZ.xml
/content/training_demo/images/train/IQ6FEAIL9Y.xml
/content/training_demo/images/train/2WA7O2SG0J.xml
/content/training_demo/images/train/4OQDM7Y542.xml
/content/training_demo/images/train/3DHCZXWSD8.xml
/cont

### Compress Dataset

Follow this step if you wish to archive your dataset for future use

In [20]:
!zip -r partitioned-dataset.zip /content/training_demo/

  adding: content/training_demo/ (stored 0%)
  adding: content/training_demo/annotations/ (stored 0%)
  adding: content/training_demo/annotations/test_labels.csv (deflated 66%)
  adding: content/training_demo/annotations/label_map.pbtxt (deflated 42%)
  adding: content/training_demo/annotations/train_labels.csv (deflated 69%)
  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/test/ (stored 0%)
  adding: content/training_demo/images/test/GMD7MLC7N3.jpg (deflated 1%)
  adding: content/training_demo/images/test/F0ZN6JPDPY.jpg (deflated 0%)
  adding: content/training_demo/images/test/AR2N1SNBBY.jpg (deflated 0%)
  adding: content/training_demo/images/test/03UNY1M83X.jpg (deflated 0%)
  adding: content/training_demo/images/test/F0ZN6JPDPY.xml (deflated 46%)
  adding: content/training_demo/images/test/3LSJ8CTMO6.xml (deflated 46%)
  adding: content/training_demo/images/test/E4B2XWUK13.jpg (deflated 0%)
  adding: content/training_demo/images/test/0KF2WT

In [21]:
!gsutil cp partitioned-dataset.zip /content/drive/My\ Drive/Datasets/

Copying file://partitioned-dataset.zip...
-
Operation completed over 1 objects/198.4 MiB.                                    
